In [60]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f, header=None)
practices.columns = col_names
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [8]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [9]:
summary = scripts[['items', 'quantity', 'nic', 'act_cost']].describe()
summary

,items,quantity,nic,act_cost
count,973193.000000,973193.000000,973193.000000,973193.000000
mean,9.133136,741.329835,73.058915,67.986613
std,29.204198,3665.426958,188.070257,174.401703
min,1.000000,0.000000,0.000000,0.040000
25%,1.000000,28.000000,7.800000,7.330000
50%,2.000000,100.000000,22.640000,21.220000
75%,6.000000,350.000000,65.000000,60.670000
max,2384.000000,577720.000000,16320.000000,15108.320000


In [10]:
summary_stats = [(col, (scripts[col].sum(), summary.loc['mean', col], summary.loc['std', col], summary.loc['25%', col], summary.loc['50%', col], summary.loc['75%', col])) for col in summary.columns]

In [11]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [12]:
item_by_bnf = scripts.groupby('bnf_name')['items'].sum()
max_item = item_by_bnf.max()

In [13]:
item_by_bnf.sort_values(ascending=False, inplace=True)
item_by_bnf = item_by_bnf.reset_index()

In [14]:
most_common_item = [(item_by_bnf.iloc[0, 0], item_by_bnf.iloc[0, 1])]
print(most_common_item)

[('Omeprazole_Cap E/C 20mg', 218583)]


In [15]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [16]:
practice_post = practices.groupby('code')['post_code'].unique().reset_index()
practice_post['post'] = practice_post['post_code'].apply(lambda list: sorted(list)[0])
practice_post.set_index('code', inplace=True)
practice_post.head()

,post_code,post
code,,
A81001,[TS18 1HU],TS18 1HU
A81002,[TS18 2AW],TS18 2AW
A81003,"[TS26 8DB, TS25 1QU]",TS25 1QU
A81004,"[TS1 3BE, TS5 8SB]",TS1 3BE
A81005,[TS14 7DJ],TS14 7DJ


In [17]:
scripts['post_code'] = scripts['practice'].apply(lambda code: practice_post.loc[code, 'post'])
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF


In [18]:
post_sum = scripts.groupby('post_code')['items'].sum().reset_index()
post_sum.head()

,post_code,items
0,B11 4BW,22731
1,B12 9LP,17073
2,B18 7AL,20508
3,B21 9RY,31027
4,B23 6DJ,28011


In [19]:
post_items = scripts.groupby(['post_code', 'bnf_name']).agg({'items': 'sum'}).reset_index().sort_values(['post_code', 'items', 'bnf_name'], ascending=[True, False, True])
post_items.head()

,post_code,bnf_name,items
1628,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706
1451,B11 4BW,Paracet_Tab 500mg,451
1217,B11 4BW,Metformin HCl_Tab 500mg,387
1093,B11 4BW,Lansoprazole_Cap 30mg (E/C Gran),385
76,B11 4BW,Amoxicillin_Cap 500mg,350


In [20]:
post_max_bnf = post_items.groupby('post_code').agg({'bnf_name': (lambda list: list.iloc[0])}).reset_index()
post_max_bnf.head()

,post_code,bnf_name
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,Paracet_Tab 500mg
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,Metformin HCl_Tab 500mg
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran)


In [21]:
post_max_items = post_items.groupby('post_code')['items'].max().reset_index()
post_max_items.head()

,post_code,items
0,B11 4BW,706
1,B12 9LP,425
2,B18 7AL,556
3,B21 9RY,1033
4,B23 6DJ,599


In [22]:
item_by_post = post_max_bnf.merge(post_max_items, on='post_code').merge(post_sum, on='post_code')
item_by_post.columns = ['post_code', 'bnf_name', 'items_max', 'items_sum']
item_by_post.head()

,post_code,bnf_name,items_max,items_sum
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,22731
1,B12 9LP,Paracet_Tab 500mg,425,17073
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,20508
3,B21 9RY,Metformin HCl_Tab 500mg,1033,31027
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,28011


In [23]:
items_by_region = [(row[0], row[1], row[2] / row[3]) for index, row in item_by_post.iterrows()]
items_by_region = items_by_region[:100]

In [24]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [25]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [26]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [27]:
chem = chem.sort_values('CHEM SUB').drop_duplicates(subset='CHEM SUB')

In [28]:
scripts = scripts.merge(chem, how='left', left_on='bnf_code', right_on='CHEM SUB')

scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,CHEM SUB,NAME
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,0106020C0,Bisacodyl
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,0106040M0,Macrogol 3350
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,0301011R0,Salbutamol
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,0304010G0,Chlorphenamine Maleate
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,0401020K0,Diazepam


In [29]:
scripts.rename(columns={'NAME': 'chem'}, inplace=True)
scripts.drop('CHEM SUB', axis=1, inplace=True)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,chem
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,Bisacodyl
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,Macrogol 3350
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,Salbutamol
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,Chlorphenamine Maleate
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,Diazepam


In [30]:
import re
scripts['opioid'] = scripts['chem'].str.contains('morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine', flags=re.I)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,chem,opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,Bisacodyl,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,Macrogol 3350,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,Salbutamol,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,Chlorphenamine Maleate,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,Diazepam,False


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [31]:
practice_sum = scripts['practice'].value_counts().reset_index()
practice_sum.columns = ['practice', 'sum']
practice_sum.head()

,practice,sum
0,N83028,2844
1,L83100,2797
2,D81043,2459
3,B81008,2396
4,B81026,2347


In [32]:
scripts['opioid'].fillna(False, inplace=True)
scripts.astype({'opioid': 'int'}, copy=False)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,chem,opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,Bisacodyl,0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,Macrogol 3350,0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,Salbutamol,0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,Chlorphenamine Maleate,0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,Diazepam,0
5,N85639,0406000T0,Prochlpzine Mal_Tab 5mg,1,0.97,0.91,28,CH44 5UF,Prochlorperazine Maleate,0
6,N85639,0407010F0,Co-Codamol_Cap 30mg/500mg,1,0.84,0.89,24,CH44 5UF,Co-Codamol (Codeine Phos/Paracetamol),1
7,N85639,0407010F0,Zapain_Tab 30mg/500mg,1,3.03,2.82,100,CH44 5UF,Co-Codamol (Codeine Phos/Paracetamol),1
8,N85639,0407010H0,Paracet_Oral Susp Paed 120mg/5ml,1,0.62,0.69,100,CH44 5UF,Paracetamol,0
9,N85639,0501011P0,Phenoxymethylpenicillin Pot_Tab 250mg,2,5.94,5.72,160,CH44 5UF,Phenoxymethylpenicillin (Penicillin V),0


In [33]:
practice_opi = scripts.groupby('practice')['opioid'].mean().reset_index()
practice_opi.head()

,practice,opioid
0,A81005,0.033179
1,A81007,0.043329
2,A81011,0.046556
3,A81012,0.042793
4,A81017,0.038140


In [34]:
practice_opi = practice_opi.merge(practice_sum, on='practice')
practice_opi.head()

,practice,opioid,sum
0,A81005,0.033179,1507
1,A81007,0.043329,1454
2,A81011,0.046556,1568
3,A81012,0.042793,1332
4,A81017,0.038140,2150


How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [35]:
opioids_mean = scripts['opioid'].mean()

In [36]:
practice_opi['relative_opi'] = practice_opi['opioid'] - opioids_mean
practice_opi.head()

,practice,opioid,sum,relative_opi
0,A81005,0.033179,1507,-0.002624
1,A81007,0.043329,1454,0.007526
2,A81011,0.046556,1568,0.010753
3,A81012,0.042793,1332,0.006990
4,A81017,0.038140,2150,0.002337


Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [37]:
std_opi = scripts['opioid'].std()

In [38]:
practice_opi['std_error'] = std_opi / ((practice_opi['sum'])**0.5)
practice_opi['opi_score'] = practice_opi['relative_opi'] / practice_opi['std_error']
practice_opi

,practice,opioid,sum,relative_opi,std_error,opi_score
0,A81005,0.033179,1507,-0.002624,0.004786,-0.548306
1,A81007,0.043329,1454,0.007526,0.004873,1.544557
2,A81011,0.046556,1568,0.010753,0.004692,2.291795
3,A81012,0.042793,1332,0.006990,0.005091,1.373060
4,A81017,0.038140,2150,0.002337,0.004007,0.583168
5,A81018,0.033431,1705,-0.002372,0.004500,-0.527080
6,A81021,0.038080,1917,0.002278,0.004244,0.536712
7,A81023,0.036752,1687,0.000949,0.004524,0.209759
8,A81029,0.037975,1264,0.002172,0.005226,0.415600
9,A81031,0.041249,1697,0.005446,0.004510,1.207583


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [39]:
practice_names = practices.groupby('code')['name'].unique().reset_index()
practice_names['pr_name'] = practice_names['name'].apply(lambda list: sorted(list)[0])
practice_names = practice_names.rename(columns={'code': 'practice'})
practice_names.head()

,practice,name,pr_name
0,A81001,[THE DENSHAM SURGERY],THE DENSHAM SURGERY
1,A81002,[QUEENS PARK MEDICAL CENTRE],QUEENS PARK MEDICAL CENTRE
2,A81003,[VICTORIA MEDICAL PRACTICE],VICTORIA MEDICAL PRACTICE
3,A81004,"[WOODLANDS ROAD SURGERY, BLUEBELL MEDICAL CENTRE]",BLUEBELL MEDICAL CENTRE
4,A81005,[SPRINGWOOD SURGERY],SPRINGWOOD SURGERY


In [40]:
practice_opi = practice_opi.merge(practice_names, on='practice')
practice_opi.drop('name', axis=1, inplace=True)
practice_opi.head()

,practice,opioid,sum,relative_opi,std_error,opi_score,pr_name
0,A81005,0.033179,1507,-0.002624,0.004786,-0.548306,SPRINGWOOD SURGERY
1,A81007,0.043329,1454,0.007526,0.004873,1.544557,BANKHOUSE SURGERY
2,A81011,0.046556,1568,0.010753,0.004692,2.291795,CHADWICK PRACTICE
3,A81012,0.042793,1332,0.006990,0.005091,1.373060,WESTBOURNE MEDICAL CENTRE
4,A81017,0.038140,2150,0.002337,0.004007,0.583168,WOODBRIDGE PRACTICE


In [41]:
practice_opi.sort_values('opi_score', ascending=False, inplace=True)
practice_opi.head()

,practice,opioid,sum,relative_opi,std_error,opi_score,pr_name
714,Y01852,0.857143,7,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
754,Y03006,1.000000,2,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
772,Y03668,0.183333,60,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
261,G81703,0.194444,36,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
819,Y04997,0.087227,321,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [42]:
opi_100 = practice_opi[:100]
opi_100.head()

,practice,opioid,sum,relative_opi,std_error,opi_score,pr_name
714,Y01852,0.857143,7,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
754,Y03006,1.000000,2,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
772,Y03668,0.183333,60,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
261,G81703,0.194444,36,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
819,Y04997,0.087227,321,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [43]:
anomalies = []
for index, row in opi_100.iterrows():
    anomalies.append((row[6], row[5], row[2]))
anomalies

[('NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('OUTREACH SERVICE NH / RH', 7.339043019238823, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60),
 ('H&R P C SPECIAL SCHEME', 5.123032414033079, 36),
 ('HMR BARDOC OOH', 4.958866438487605, 321),
 ('INTEGRATED CARE 24 LTD (CWSX OOH)', 4.8888781604828235, 426),
 ('DARWEN HEALTHCARE', 4.8391589686363385, 1917),
 ('THE LIMES MEDICAL PRACTICE', 4.546841872334426, 1321),
 ('IC24 LTD (BRIGHTON & HOVE OOH)', 4.335047010605197, 357),
 ('OLDHAM 7 DAY ACCESS HUB2 OOH', 4.31178403661019, 56),
 ('IC24 LTD (NORFOLK & WISBECH OOH)', 4.2575005924727645, 489),
 ('ROSSENDALE MIU & OOH', 4.256827446322491, 18),
 ('BURY WALK-IN CENTRE', 4.150589122881536, 138),
 ('IC24 LTD (HORSHAM & MID SUSSEX OOH)', 3.7816207038443523, 215),
 ('LCW HOUNSLOW CCG OOH', 3.582848546206269, 69),
 ('WEEKEND WORKING EASINGTON NORTH', 3.565938771129764, 278),
 ('COMPASS ENFIELD', 3.5587202651067935, 7),
 ('BASSETLAW DRUG & ALCOHOL SERVICE', 3.53326410251154

In [44]:
grader.score.dw__script_anomalies(anomalies)

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [45]:
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts16 = pd.read_csv(f)
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [46]:
bnf16 = scripts16['bnf_name'].value_counts().reset_index()
bnf16.columns = ['bnf_name', 'count16']
bnf16.head()

,bnf_name,count16
0,GlucoRX FinePoint Needles Pen Inj Screw,1532
1,3m Health Care_Cavilon Durable Barrier C,825
2,Fluclox Sod_Cap 500mg,796
3,Amoxicillin_Cap 500mg,790
4,Prednisolone_Tab 5mg,786


In [47]:
bnf17 = scripts['bnf_name'].value_counts().reset_index()
bnf17.columns = ['bnf_name', 'count17']
bnf17.head()

,bnf_name,count17
0,GlucoRX FinePoint Needles Pen Inj Screw,1718
1,3m Health Care_Cavilon Durable Barrier C,816
2,Prednisolone_Tab 5mg,785
3,Fluclox Sod_Cap 500mg,783
4,Amoxicillin_Cap 500mg,777


In [48]:
growth = bnf16.merge(bnf17, how='inner', on='bnf_name')
growth.fillna(0, inplace=True)
growth.head()

,bnf_name,count16,count17
0,GlucoRX FinePoint Needles Pen Inj Screw,1532,1718
1,3m Health Care_Cavilon Durable Barrier C,825,816
2,Fluclox Sod_Cap 500mg,796,783
3,Amoxicillin_Cap 500mg,790,777
4,Prednisolone_Tab 5mg,786,785


In [49]:
growth['growth_rate'] = (growth['count17'] - growth['count16']) / growth['count16']
growth.sort_values('growth_rate', ascending=False, inplace=True)
growth.head()

,bnf_name,count16,count17,growth_rate
10738,Vensir XL_Cap 225mg,1,93,92.000
9639,Fludroxycortide_Tape 7.5cm x 20cm,2,106,52.000
11157,Tamiflu_Cap 75mg,1,44,43.000
7157,Enstilar_Foam Aero 50mcg/0.5mg/g,8,225,27.125
10980,Memantine HCl_Orodisper Tab 10mg S/F,1,28,27.000


In [50]:
growth = growth[growth['count16'] >= 50]

In [51]:
script_growth = []
for index, row in growth.head(50).iterrows():
    script_growth.append((row[0], row[3], row[1]))
for index, row in growth.tail(50).iterrows():
    script_growth.append((row[0], row[3], row[1]))
script_growth

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86),
 ('Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134716, 193),
 ('Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692, 52),
 ('Trulicity_Inj 1.5mg/0.5ml Pf Pen', 1.1851851851851851, 54),
 ('CosmoCol_Paed Oral Pdr Sach 6.9g', 1.1774193548387097, 62),
 ('Dulaglutide_Inj 1.5mg/0.5ml Pf Dev', 0.9634146341463414, 82),
 ('ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs', 0.912, 125),
 ('Empagliflozin_Tab 25mg', 0.896, 125),
 ('CareSens Lancets 0.31mm/30 Gauge', 0.8955223880597015, 67),
 ('Fostair_Inh 200mcg/6mcg (120D) CFF', 0.8616600790513834, 253),
 ('Orbis Nor Saline Sod Chlor 0.9% Nsl Dps', 0.8588235294117647, 85),
 ('Umeclidinium Brom_Inh 65mcg (30D)', 0.8275862068965517, 87),
 ('Ultibro Breezhaler_Pdr Inh Cap + Dev', 0.8016528925619835, 121),
 ('Medihoney_Barrier Crm', 0.7840909090909091, 88),
 ('Fluox

In [52]:
grader.score.dw__script_growth(script_growth)

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [111]:
p = 1 / scripts['bnf_code'].nunique()
p

0.0005063291139240507

In [112]:
rates = scripts['bnf_code'].value_counts() / scripts.shape[0]

In [113]:
scripts['rate'] = scripts['bnf_code'].apply(lambda code: rates[code])
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,chem,opioid,rare,p,rate
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,Bisacodyl,False,1,0.001064,0.001064
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,Macrogol 3350,False,1,0.005002,0.005002
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,Salbutamol,False,1,0.007054,0.007054
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,Chlorphenamine Maleate,False,1,0.002200,0.002200
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,Diazepam,False,1,0.002278,0.002278


In [114]:
scripts.drop('p', axis=1, inplace=True)
scripts['rare'] = scripts['rate'] < 0.1 * p
scripts['rare'] = scripts['rare'].astype(int)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,chem,opioid,rare,rate
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,Bisacodyl,False,0,0.001064
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,Macrogol 3350,False,0,0.005002
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,Salbutamol,False,0,0.007054
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,Chlorphenamine Maleate,False,0,0.002200
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,Diazepam,False,0,0.002278


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [140]:
rare_cost_prop = scripts[scripts['rare'] == 1].groupby('practice')['act_cost'].sum() / scripts.groupby('practice')['act_cost'].sum()
rare_cost_prop

practice
A81005    0.012017
A81007    0.008381
A81011    0.005116
A81012    0.013747
A81017    0.007359
            ...   
Y05570         NaN
Y05583         NaN
Y05597         NaN
Y05660    0.000323
Y05670         NaN
Name: act_cost, Length: 856, dtype: float64

In [141]:
rare_cost_prop.fillna(0, inplace=True)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [142]:
relative_rare_cost_prop = rare_cost_prop - scripts[scripts['rare'] == 1]['act_cost'].sum() / scripts['act_cost'].sum()
relative_rare_cost_prop

practice
A81005   -0.003946
A81007   -0.007582
A81011   -0.010847
A81012   -0.002216
A81017   -0.008604
            ...   
Y05570   -0.015963
Y05583   -0.015963
Y05597   -0.015963
Y05660   -0.015640
Y05670   -0.015963
Name: act_cost, Length: 856, dtype: float64

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [143]:
std_error = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [144]:
rare_scores = relative_rare_cost_prop / std_error
rare_scores = rare_scores.reset_index()
rare_scores.columns=['practice', 'z_score']
rare_scores.head()

,practice,z_score
0,A81005,-0.065216
1,A81007,-0.125308
2,A81011,-0.179263
3,A81012,-0.036615
4,A81017,-0.142190


In [145]:
rare_scores = rare_scores.merge(practice_opi[['practice', 'pr_name']], on='practice')
rare_scores.head()

,practice,z_score,pr_name
0,A81005,-0.065216,SPRINGWOOD SURGERY
1,A81007,-0.125308,BANKHOUSE SURGERY
2,A81011,-0.179263,CHADWICK PRACTICE
3,A81012,-0.036615,WESTBOURNE MEDICAL CENTRE
4,A81017,-0.142190,WOODBRIDGE PRACTICE


In [103]:
# practice_post = practice_post.reset_index()
# practice_post.drop('post_code', axis=1, inplace=True)
# practice_post.columns = ['practice', 'post']
# practice_post.columns = ['practice', 'post']
practice_post.head()

,practice,post
0,A81001,TS18 1HU
1,A81002,TS18 2AW
2,A81003,TS25 1QU
3,A81004,TS1 3BE
4,A81005,TS14 7DJ


In [146]:
rare_scores = rare_scores.merge(practice_post, on='practice')
rare_scores.head()

,practice,z_score,pr_name,post
0,A81005,-0.065216,SPRINGWOOD SURGERY,TS14 7DJ
1,A81007,-0.125308,BANKHOUSE SURGERY,TS24 7PW
2,A81011,-0.179263,CHADWICK PRACTICE,TS24 7PW
3,A81012,-0.036615,WESTBOURNE MEDICAL CENTRE,TS3 6AL
4,A81017,-0.142190,WOODBRIDGE PRACTICE,TS17 0EE


In [147]:
rare_scores.sort_values('z_score', ascending=False, inplace=True)
rare_scores.head()

,practice,z_score,pr_name,post
765,Y03472,16.262687,CONSULTANT DIABETES TEAM,PE1 2QP
831,Y05320,15.128648,DMC COMMUNITY DERMATOLOGY RBWF,B61 0AZ
793,Y04404,7.542139,OUTPATIENTS JUBILEE HEALTH CENTRE,SM6 0HY
766,Y03484,7.287222,DMC COMMUNITY DERMATOLOGY CLINIC,ME12 3LT
794,Y04424,6.838614,DMC HEALTHCARE,SS8 0JA


In [150]:
rare_scripts = [(row[0], row[2], row[1]) for index, row in rare_scores.head(100).iterrows()]
rare_scripts

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655073),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416869),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.54213935610462),
 ('Y03484', 'DMC COMMUNITY DERMATOLOGY CLINIC', 7.287222200297828),
 ('Y04424', 'DMC HEALTHCARE', 6.838614181432866),
 ('Y00631', 'BINGLEY DERMATOLOGY CLINIC', 5.75599754983774),
 ('A89038', 'BARMSTON MEDICAL CENTRE', 5.664940368539935),
 ('Y01696', 'BASSETLAW HOSPICE OF THE GOOD SHEPHERD', 4.290685171474921),
 ('Y03699', 'OLDHAM DERMATOLOGY SERVICE', 4.103743999157244),
 ('Y02045', 'VERNOVA HEALTHCARE CIC', 3.1807154704143983),
 ('Y02823', 'DMC VICARAGE LANE', 3.054121649498702),
 ('Y00997', 'COMMUNITY DERMATOLOGY SERVICE', 3.018128359535277),
 ('Y05019', 'OLDHAM TOTAL SKIN SERVICE', 2.7247515958989283),
 ('Y01003', 'BURY OOH', 2.350436347139858),
 ('Y05431', 'NOTTS APPLIANCE MANAGEMENT SERVICE', 2.0371585432333865),
 ('Y03557', 'IPU 10-17 RECOVERY COVENTRY', 2.0304495001187792),
 ('Y05419', 

In [151]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*